In [2]:
import arcpy, os, sys, shutil
from arcpy import env

In [3]:
env.overwriteOutput = True

In [4]:
#Set environment settings
env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"

In [6]:
input_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [10]:
#Set local variables
inNetworkDataset = "Network_Ped_Bike/Network_ND"

In [4]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/network-analyst/build-network.htm
try:
    #Check out Network Analyst license if available. Fail if the Network Analyst license is not available.
    if arcpy.CheckExtension("network") == "Available":
        arcpy.CheckOutExtension("network")
    else:
        raise arcpy.ExecuteError("Network Analyst Extension license is not available.")
    
    #Build the network dataset
    arcpy.BuildNetwork_na(inNetworkDataset)
    
    #If there are any build errors, they are recorded in a BuildErrors.txt file
    #present in the system temp directory. So copy this file to the directory
    #containing this script
    
    #First get the path to the system temp directory
    tempDir = os.environ.get("TEMP")
    if tempDir:
        shutil.copy2(os.path.join(tempDir,"BuildErrors.txt"),sys.path[0])
    
    print("Script completed successfully.")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully.


In [12]:
layer_name = "Service Area"

In [13]:
impedance = "Biking by distance"

In [14]:
dist = 17582.4

In [15]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/network-analyst/make-service-area-layer.htm
#Create a new service area layer.
result_object = arcpy.na.MakeServiceAreaAnalysisLayer(inNetworkDataset, layer_name, impedance, 
                                                      "FROM_FACILITIES", [dist], None, "LOCAL_TIME_AT_LOCATIONS", 
                                                      "POLYGONS", "STANDARD", "OVERLAP", "RINGS", "100 Meters", None, None)

In [16]:
#Get the layer object from the result object. The service area layer can now be referenced using the layer object.
layer_object = result_object.getOutput(0)

In [17]:
#Get the names of all the sublayers within the service area layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)

In [18]:
sublayer_names

{'Facilities': 'Facilities',
 'Barriers': 'Point Barriers',
 'SALines': 'Lines',
 'PolylineBarriers': 'Line Barriers',
 'SAPolygons': 'Polygons',
 'PolygonBarriers': 'Polygon Barriers'}

In [19]:
#Stores the layer names that we will use later
facilities_layer_name = sublayer_names["Facilities"]
polygons_layer_name = sublayer_names["SAPolygons"]

In [20]:
facilities = os.path.join(input_folder, "PerformanceAnalysis", "service_transit_equity", "service_stops.shp")

In [21]:
field_mappings = arcpy.na.NAClassFieldMappings(layer_object, facilities_layer_name)

In [22]:
#Load the fire stations as facilities.
arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities,
                        field_mappings, "", exclude_restricted_elements = "EXCLUDE")

<Result 'GPLYR_{DE19E4AF-CA82-42AC-8328-E33B4DDB8D85}'>

In [23]:
#Get sublayers to work with later
facilities_sublayer = layer_object.listLayers(facilities_layer_name)[0]
polygons_sublayer = layer_object.listLayers(polygons_layer_name)[0]

In [24]:
#Get the Service Area Layer's solver properties. This can be used to set individual properties later without 
#re-creating the layer.
solver_properties = arcpy.na.GetSolverProperties(layer_object)

In [25]:
#Solve the service area layer
arcpy.na.Solve(layer_object)

id,value
0,a Layer object
1,true


In [26]:
#Transfer the FireStationID field from the input Facilities to the output Polygons
arcpy.management.AddJoin(polygons_sublayer, "FacilityID", facilities_sublayer, "ObjectID")

<Result 'GPLYR_{D5F35EEF-98E5-424C-8716-5F0993ED01D4}'>

In [27]:
mode = 'Biking'

In [30]:
out_featureclass = "SA" + mode

In [31]:
#Append the polygons to the output feature class. If this was the first solve, create the feature class.
if not arcpy.Exists(out_featureclass):
    arcpy.management.CopyFeatures(polygons_sublayer, out_featureclass)
else:
    arcpy.management.Append(polygons_sublayer, out_featureclass)

In [34]:
# function to get the service area polygons
def ServiceAreaAnalysis(layer_name = "Service Area", 
                       impedance = "Biking by distance",
                       dist = 17582.4,
                       facilities = os.path.join(input_folder, "PerformanceAnalysis", 
                                                 "service_transit_equity", "service_stops.shp"),
                       mode = 'Biking'):
    print('Creating service area analysis layer...')
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(inNetworkDataset, layer_name, impedance, 
                                                      "FROM_FACILITIES", [dist], None, "LOCAL_TIME_AT_LOCATIONS", 
                                                      "POLYGONS", "STANDARD", "OVERLAP", "RINGS", "100 Meters", None, None)
    print('Done...')
    layer_object = result_object.getOutput(0)
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    facilities_layer_name = sublayer_names["Facilities"]
    polygons_layer_name = sublayer_names["SAPolygons"]
    print('Adding facilities...')
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,
                                                    facilities_layer_name)
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities,
                        field_mappings, "", exclude_restricted_elements = "EXCLUDE")
    print('Done...')
    facilities_sublayer = layer_object.listLayers(facilities_layer_name)[0]
    polygons_sublayer = layer_object.listLayers(polygons_layer_name)[0]
    print('Sovling service area...')
    solver_properties = arcpy.na.GetSolverProperties(layer_object)
    arcpy.na.Solve(layer_object)
    print('Done...')
    print('Exporting service area polygons...')
    arcpy.management.AddJoin(polygons_sublayer, "FacilityID", facilities_sublayer, "ObjectID")
    out_featureclass = "SA" + mode
    if arcpy.Exists(out_featureclass):
        arcpy.Delete_management(out_featureclass)
    if not arcpy.Exists(out_featureclass):
        arcpy.management.CopyFeatures(polygons_sublayer, out_featureclass)
    else:
        arcpy.management.Append(polygons_sublayer, out_featureclass)
    print('Done...')

In [35]:
ServiceAreaAnalysis()

Creating service area analysis layer...
Done...
Adding facilities...
Done...
Sovling service area...
Done...
Exporting service area polygons...
Done...
